In [128]:
import pandas as pd
import numpy as np
import time
import string
from unidecode import unidecode

In [129]:
df_lyrics = pd.read_csv("data/lyrics.csv", sep =';')

df_lyrics[:5]

,ALink,SName,SLink,Lyric,ParentGenre,Genres,Genres_API
0,/andre-drake/,Bae ft. V.P Rap,/andre-drake/bae-ft-v-p-rap.html,[André Drake]. Minha princesa eu troco tudo pr...,Black Music,"Black Music, Hip Hop, Rap","Black Music, Hip Hop, Rap"
1,/andre-drake/,By Chance (Brazilian Remix),/andre-drake/by-chance-brazilian-remix.html,"[Refrão]. Mais views, pode ser?. Cê sabe, que ...",Black Music,"Black Music, Hip Hop, Rap","Black Music, Hip Hop, Rap"
2,/andre-drake/,Casa do Terror,/andre-drake/casa-do-terror.html,"4 Paredes uma jaula, na minha mão uma faca. Um...",Black Music,"Black Music, Hip Hop, Rap","Black Music, Hip Hop, Rap"
3,/andre-drake/,Na Madrugada,/andre-drake/na-madrugada.html,"[Refrão 1]. Na ""madruga"" não consigo dormir. m...",Black Music,"Black Music, Hip Hop, Rap","Black Music, Hip Hop, Rap"
4,/andre-drake/,V.P Convida Parte 2,/andre-drake/v-p-convida-parte-2.html,[Vp Rap]. Vp de novo que convida. Dessa vez se...,Black Music,"Black Music, Hip Hop, Rap","Black Music, Hip Hop, Rap"


In [130]:
# Unindo estilos
dict_map_estilos = {
    "Pagode": "Pagode_Samba",
    "Samba": "Pagode_Samba",
    "Regional": "Regional_Sertanejo_Forró_Country",
    "Sertanejo": "Regional_Sertanejo_Forró_Country",
    "Forró": "Regional_Sertanejo_Forró_Country",
    "Country": "Regional_Sertanejo_Forró_Country",
    "Rock": "Rock_Pop/Rock_Rock Alternativo",
    "Pop/Rock": "Rock_Pop/Rock_Rock Alternativo",
    "Rock Alternativo" : "Rock_Pop/Rock_Rock Alternativo",
    "Rap": "Rap_Hip Hop",
    "Hip Hop": "Rap_Hip Hop",
    "MPB": "MPB_Bossa_Nova",
    "Bossa Nova": "MPB_Bossa_Nova"
}

df_lyrics = df_lyrics.replace({"ParentGenre": dict_map_estilos})

serie_parent_genre = df_lyrics["ParentGenre"].value_counts()

In [131]:
# Filtrando dataset de músicas para deixar apenas gêneros com mais de 1500 letras musicais.
generos_remover = ['Infantil', 'Romântico', 'Velha Guarda']
generos_selecionados = list(serie_parent_genre[serie_parent_genre >3000].index)
generos_selecionados = [item for item in generos_selecionados if item not in generos_remover]
df_lyrics = df_lyrics[df_lyrics["ParentGenre"].isin(generos_selecionados)]

df_lyrics["ParentGenre"].value_counts()

Regional_Sertanejo_Forró_Country    37659
Gospel/Religioso                    33586
MPB_Bossa_Nova                      15701
Pagode_Samba                        11594
Rock_Pop/Rock_Rock Alternativo      11031
Rap_Hip Hop                          6189
Funk Carioca                         4209
Pop                                  3535
Name: ParentGenre, dtype: int64

In [132]:
# Removendo músicas instrumentais
df_lyrics = df_lyrics[ ~((df_lyrics.Lyric.str.contains("Instrumental")) & \
                         (df_lyrics.Lyric.str.split(' ').map(lambda x: len(x)<20))) ]\
                     .reset_index(drop=True)

In [133]:
# limit é uma variável para determinar quantas letras devem ser utilizadas para o processamento
# útil apenas para debug, caso contrário utilizar o tamanho total do dataset
limit = df_lyrics["Lyric"].shape[0]
serie_lyrics = df_lyrics["Lyric"][0:limit]

In [134]:
# Removendo conteúdo [] e ()
# Transformando strings em minúsculo
# Removendo pontuação 
# Removendo notas de rodapé

def process_lyric(lyric):
    try:
        #lyric = unidecode(lyric)
        lyric = lyric.translate(str.maketrans('', '', string.punctuation))
    except:
        pass
    return lyric

start = time.time()  
serie_lyrics = serie_lyrics.str.lower()
serie_lyrics = serie_lyrics.str.replace('\-\-(.*?)\-\-',' ')
serie_lyrics = serie_lyrics.str.replace('\=\=(.*?)\=\=',' ')
serie_lyrics = serie_lyrics.str.replace('\*\*(.*?)\*\*',' ')
serie_lyrics = serie_lyrics.str.replace("[\(\[].*?[\)\]]"," ")
serie_lyrics = serie_lyrics.str.replace("refrão"," ")
serie_lyrics = serie_lyrics.str.replace("2x"," ")
serie_lyrics = serie_lyrics.map(process_lyric)

def processar_array_split_rodape(x):
    if(len(x)>1):
        x = x[:-1]
    return ''.join(x)

serie_lyrics = serie_lyrics.str.split("enviado por").map(processar_array_split_rodape)
serie_lyrics = serie_lyrics.str.split("adicionado por").map(processar_array_split_rodape)
serie_lyrics = serie_lyrics.str.split("submited by").map(processar_array_split_rodape)


end = time.time()  
print('\nTime(s): '+str("%.2f" % (end - start)+'s\n'))


Time(s): 9.60s



In [137]:
# Criando e salvando train_data e test_data. (Proporção 80/20) 
# Tais arquivos serão utilizados pelo pytext para treino e avaliação do resultado.

df_all = pd.DataFrame({"Genre": df_lyrics["ParentGenre"][0:limit], "Lyric": serie_lyrics})

df_all["Genre"] = df_all["Genre"].str.replace(" ","_")

df_all = df_all[df_all['Lyric'].str.len()>0]

msk = np.random.rand(len(df_all)) < 0.8

df_train = df_all[msk]
df_test  = df_all[~msk]

df_train.to_csv('train_data.tsv', sep='\t', index = False, header = False)
df_test.to_csv('test_data.tsv', sep='\t' , index = False, header = False)

In [15]:
dict_series = df_all["Genre"].value_counts().to_dict()
total = df_lyrics.shape[0]
for key in dict_series.keys():
    dict_series[key] = round(total*100/(100*dict_series[key]),1)
dict_series

{'Regional_Sertanejo_Forró_Country': 3.3,
 'Gospel/Religioso': 3.7,
 'MPB_Bossa_Nova': 7.9,
 'Pagode_Samba': 10.7,
 'Rock_Pop/Rock_Rock_Alternativo': 11.2,
 'Rap_Hip_Hop': 20.0,
 'Funk_Carioca': 29.3,
 'Pop': 34.9}